In [8]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import sklearn as sk
import datasets as ds

## The dataset

### Question 1

In [20]:
splits = ds.get_dataset_split_names('imdb')
print(splits)
print(f'Number of splits: {len(splits)}')

['train', 'test', 'unsupervised']
Number of splits: 3


There are 3 splits in the IMDB dataset.

### Question 2

In [21]:
dataset_0 = ds.load_dataset('imdb', split=splits[0])
dataset_1 = ds.load_dataset('imdb', split=splits[1])
dataset_2 = ds.load_dataset('imdb', split=splits[2])

Found cached dataset imdb (/Users/francois.soulier/.cache/huggingface/datasets/imdb/plain_text/1.0.0/d613c88cf8fa3bab83b4ded3713f1f74830d1100e171db75bbddb80b3345c9c0)
Found cached dataset imdb (/Users/francois.soulier/.cache/huggingface/datasets/imdb/plain_text/1.0.0/d613c88cf8fa3bab83b4ded3713f1f74830d1100e171db75bbddb80b3345c9c0)
Found cached dataset imdb (/Users/francois.soulier/.cache/huggingface/datasets/imdb/plain_text/1.0.0/d613c88cf8fa3bab83b4ded3713f1f74830d1100e171db75bbddb80b3345c9c0)


In [23]:
print(f'{splits[0]} split size : {dataset_0.num_rows}')
print(f'{splits[1]} split size : {dataset_1.num_rows}')
print(f'{splits[2]} split size : {dataset_2.num_rows}')

train split size : 25000
test split size : 25000
unsupervised split size : 50000


In [38]:
# Dataset 0: Train
df_0 = dataset_0.to_pandas()
print("Train")
print(df_0.where(df_0['label'] == 0).count())
print(df_0.where(df_0['label'] == 1).count())

print("\n")
# Dataset 1: Test
print("Test")
df_1 = dataset_1.to_pandas()
print(df_1.where(df_1['label'] == 0).count())
print(df_1.where(df_1['label'] == 1).count())

Train
text     12500
label    12500
dtype: int64
text     12500
label    12500
dtype: int64


Test
text     12500
label    12500
dtype: int64
text     12500
label    12500
dtype: int64


Hence, each class represents 50% of the supervised dataset (both in train and test samples).